In [2]:
import numpy as np
import pandas as pd

In [32]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

def predict(col1, col2):
    train = pd.read_csv('data/train.csv',
                    usecols=[col1, col2,'hotel_cluster'],
                    chunksize=1000000)

    aggs = []
    print('-'*38)
    for chunk in train:
        agg = chunk.groupby([col2,
                             'hotel_cluster'])[col1].agg(['sum','count'])
        agg.reset_index(inplace=True)
        aggs.append(agg)
        print('.',end='')
    print('')
    aggs = pd.concat(aggs, axis=0)
    
    CLICK_WEIGHT = 0.05
    agg = aggs.groupby([col2,'hotel_cluster']).sum().reset_index()
    agg['count'] -= agg['sum']
    agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
    agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
    
    most_pop = agg.groupby([col2]).apply(most_popular)
    most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
    
    test = pd.read_csv('data/test.csv',
                    usecols=[col2],)
    
    test = test.merge(most_pop, how='left',left_on=col2,right_index=True)
    
    test.hotel_cluster.isnull().sum()
    
    most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
    most_pop_all = np.array_str(most_pop_all)[1:-1]
    
    test.hotel_cluster.fillna(most_pop_all,inplace=True)
    
    test.hotel_cluster.to_csv('{},{}.csv'.format(col1,col2),header=True, index_label='id')

In [33]:
predict('is_booking', 'hotel_market')

--------------------------------------
......................................
